In [ ]:
import pandas as pd
import re
import numpy as np
import dill

from tqdm import tqdm
from collections import defaultdict, Counter

import warnings
warnings.filterwarnings("ignore")

# MIMIC

In [ ]:
PATIENTS = pd.read_csv('/hosp/PATIENTS.csv.gz')
ADMISSIONS = pd.read_csv('/hosp/ADMISSIONS.csv.gz')
ICUSTAYS = pd.read_csv('/ICUSTAYS.csv.gz')

ADMISSIONS.columns = ADMISSIONS.columns.str.upper()
ICUSTAYS.columns = ICUSTAYS.columns.str.upper()
PATIENTS.columns = PATIENTS.columns.str.upper()

In [ ]:
def MIMICiv(ADMISSIONS,ICUSTAYS,PATIENTS):
    # Select relevant columns for ADMISSIONS, ICUSTAYS, and PATIENTS
    ADMISSIONS = ADMISSIONS[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'RACE']]
    ICUSTAYS = ICUSTAYS[['HADM_ID', 'ICUSTAY_ID', 'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'INTIME', 'OUTTIME', 'LOS']]
    PATIENTS['DOB'] = PATIENTS['ANCHOR_YEAR'] - PATIENTS['ANCHOR_AGE']
    PATIENTS = PATIENTS[['SUBJECT_ID', 'GENDER', 'DOB', 'DOD','ANCHOR_YEAR_GROUP']]
    
    # Merge the datasets
    df = pd.merge(PATIENTS, ADMISSIONS, on='SUBJECT_ID', how='left')
    df = pd.merge(df, ICUSTAYS, on='HADM_ID', how='left')
    print(df.shape,len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()))
    
    # Convert date columns to datetime
    date_columns = ['ADMITTIME', 'DISCHTIME', 'INTIME', 'OUTTIME','DOD']
    df[date_columns] = df[date_columns].apply(pd.to_datetime)

    # Sort the dataframe by 'SUBJECT_ID' and 'INTIME'
    df = df.dropna(subset=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ADMITTIME', 'DISCHTIME', 'INTIME', 'OUTTIME','DOB'])
    print('Drop time missing', df.shape, len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()))
    df = df.sort_values(by=['SUBJECT_ID', 'INTIME'])

    # Create 'DOB' as a complete date by assuming January 1st for each year
    df['DOB'] = pd.to_datetime(df['DOB'].astype(str) + '-01-01')
    # Calculate 'AGE' based on 'ADMITTIME' and 'DOB', and adjust for ages above 89
    df['AGE'] = (df['ADMITTIME'] - df['DOB']).dt.days // 365.242
    df['AGE'] = df['AGE'].apply(lambda x: 91 if x > 89 else x)
    # Filter out patients under 18
    df = df[df['AGE'] >= 18]
    print('AGE >= 18',df.shape, len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()))
    
    # Select specific columns for the final dataframe
    df = df[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ADMITTIME', 'INTIME', 'DISCHTIME', 'OUTTIME', 
             'GENDER', 'DOB', 'DOD', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'RACE', 
             'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'LOS', 'AGE']]
    # Create a flag for the first admission for each patient
    df['FIRST_HADM'] = df.groupby('SUBJECT_ID')['ADMITTIME'].transform(lambda x: x == x.min()).astype(int)
    # Create a flag for the first ICU stay for each patient-HADM combination
    df['FIRST_ICU'] = df.groupby(['SUBJECT_ID', 'HADM_ID'])['INTIME'].transform(lambda x: x == x.min()).astype(int)
    # keep first ICU
    df = df[df['FIRST_ICU'] == 1]
    print('FIRST_ICU', df.shape, len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()))
    
    # Keep rows with LOS greater than 1; 
    df = df[df['LOS'] >= 1]
    print('LOS', df.shape, len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()))

    df['DIEINHOSPITAL'] = ((df['ADMITTIME'] <= df['DOD']) & (df['DOD'] <= df['DISCHTIME'])).astype(int)
    df['DIEINICU'] = ((df['INTIME'] <= df['DOD']) & (df['DOD'] <= df['OUTTIME'])).astype(int)

    df['NEXT_ADMIT'] = df.groupby('SUBJECT_ID')['ADMITTIME'].shift(-1)
    df['DAYS_TO_READMIT'] = (df['NEXT_ADMIT'] - df['ADMITTIME']).dt.days
    df['Readmission_30'] = df['DAYS_TO_READMIT'].apply(lambda x: 1 if pd.notnull(x) and x <= 30 else 0)
    df['Readmission_60'] = df['DAYS_TO_READMIT'].apply(lambda x: 1 if pd.notnull(x) and x <= 60 else 0)

    df['HOURS_FROM_ADMIT'] = (df['INTIME'] - df['ADMITTIME']).dt.total_seconds() / 3600
    df['ICU_within_12hr_of_admit'] = df['HOURS_FROM_ADMIT'].apply(lambda x: 1 if 0 <= x <= 12 else 0)

    ICUSTAYS = ICUSTAYS[ICUSTAYS.HADM_ID.isin(df.HADM_ID)]
    ICUSTAYS['Multiple_ICUs'] = (ICUSTAYS.groupby('HADM_ID')['ICUSTAY_ID']
                    .transform('count')
                    .gt(1)  
                    .astype(int)) 
    ICUSTAYS = ICUSTAYS[['HADM_ID','Multiple_ICUs']].drop_duplicates(keep='first')
    df = pd.merge(df,ICUSTAYS,on='HADM_ID',how='left')
    df.drop(columns=['NEXT_ADMIT', 'DAYS_TO_READMIT'], inplace=True)

    df['SUBJECT_ID'] = df['SUBJECT_ID'].astype(int)
    df['HADM_ID'] = df['HADM_ID'].astype(int)
    df['ICUSTAY_ID'] = df['ICUSTAY_ID'].astype(int)

    print('------------------------------------------------------------------------')
    print('Final Data：',df.shape, 'SUBJECT_ID:',len(df.SUBJECT_ID.unique()),'HADM_ID:',len(df.HADM_ID.unique()), 'ICUSTAY_ID:',len(df.ICUSTAY_ID.unique()))
    print('DIEINHOSPITAL:',Counter(df['DIEINHOSPITAL']))
    print('DIEINICU:',Counter(df['DIEINICU']))
    print('Readmission_30:',Counter(df['Readmission_30']))
    print('Readmission_60:',Counter(df['Readmission_60']))
    print('ICU_within_12hr_of_admit:',Counter(df['ICU_within_12hr_of_admit']))
    print('Multiple_ICUs:',Counter(df['Multiple_ICUs']))

    return df

In [ ]:
ADMISSIONS = ADMISSIONS.rename(columns={'ETHNICITY':'RACE'})
ICUSTAYS = ICUSTAYS.rename(columns={'STAY_ID':'ICUSTAY_ID'})

df = MIMICiv(ADMISSIONS,ICUSTAYS,PATIENTS)

In [ ]:
df.head(2)

In [ ]:
df.columns[df.isnull().any()]

In [ ]:
df.GENDER.value_counts()

In [ ]:
df_female = df[df.GENDER == 'F']
print('---------------------------Female-------------------------------------')
print('Final  ———— ：',df_female.shape, 'SUBJECT_ID:',len(df_female.SUBJECT_ID.unique()),'HADM_ID:',len(df_female.HADM_ID.unique()), 'ICUSTAY_ID:',len(df_female.ICUSTAY_ID.unique()))
print('DIEINHOSPITAL:',Counter(df_female['DIEINHOSPITAL']))
print('DIEINICU:',Counter(df_female['DIEINICU']))
print('Readmission_30:',Counter(df_female['Readmission_30']))
print('Readmission_60:',Counter(df_female['Readmission_60']))
print('ICU_within_12hr_of_admit:',Counter(df_female['ICU_within_12hr_of_admit']))
print('Multiple_ICUs:',Counter(df_female['Multiple_ICUs']))

In [ ]:
D_ICD_DIAGNOSES = pd.read_csv('/mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz')
D_ICD_DIAGNOSES.columns = D_ICD_DIAGNOSES.columns.str.upper()
D_ICD_DIAGNOSES['ICD_VERSION'] = 9
D_ICD_DIAGNOSES = D_ICD_DIAGNOSES[['ICD9_CODE','ICD_VERSION','LONG_TITLE']]
D_ICD_DIAGNOSES.columns = ['ICD_CODE', 'ICD_VERSION', 'ICD_TEXT']
print(D_ICD_DIAGNOSES.shape)

D_4_DIAGNOSES = pd.read_csv('/3.1/hosp/D_ICD_DIAGNOSES.csv.gz')
D_4_DIAGNOSES.columns = D_4_DIAGNOSES.columns.str.upper()
D_4_DIAGNOSES.columns = ['ICD_CODE', 'ICD_VERSION', 'ICD_TEXT']
print(D_4_DIAGNOSES.shape)

D_ICD_DIAGNOSES = pd.concat([D_ICD_DIAGNOSES,D_4_DIAGNOSES])
D_ICD_DIAGNOSES = D_ICD_DIAGNOSES.drop_duplicates(keep='first')
print(D_ICD_DIAGNOSES.shape)

diag_9_10 = pd.read_csv('/0_diag_9_10.csv')
diag_9_10.columns = ['ICD_CODE', 'ROOT']
diag_9_10 = diag_9_10[~diag_9_10.duplicated(subset=['ICD_CODE'], keep=False)]
diag_9_10 = diag_9_10.dropna(subset=['ROOT'])
print(diag_9_10.shape,len(diag_9_10.ICD_CODE.unique()))

In [ ]:
D_ICD_9 = D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_VERSION == 9]
D_ICD_10 = D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_VERSION == 10]
diag_9 = D_ICD_9[D_ICD_9.ICD_CODE.isin(diag_9_10.ICD_CODE)].ICD_CODE.values
diag_10 = D_ICD_10[D_ICD_10.ICD_CODE.isin(diag_9_10.ICD_CODE)].ICD_CODE.values
print(len(diag_9), len(diag_10))
intersection = set(diag_9) & set(diag_10)
print(len(intersection), intersection)

In [ ]:
diag_10 = list(set(diag_10)-set(intersection))
print(len(diag_10))

In [ ]:
only_diag_9_10 = diag_9_10[diag_9_10.ICD_CODE.isin(diag_9)]
only_diag_9_10.shape

In [ ]:
del(diag_9_10)

In [ ]:
check_AKI_9 = ['5845', '5846', '5847', '5848','5849','66930','66932','66934']
check_AKI_10 = ['N17','N170','N171','N172','N179']
AKI_ids = list(check_AKI_9 + check_AKI_10)

check_CKD_9 = ['5851', '5852', '5853', '5854', '5855','5859'] 
check_CKD_10 = ['N18','N181','N182', 'N183', 'N1830', 'N1831', 'N1832', 'N184', 'N185', 'N189']
CKD_ids = list(check_CKD_9 + check_CKD_10)

In [ ]:
specific_need = list(pd.unique(AKI_ids + CKD_ids))
print(len(specific_need),len(D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE.isin(specific_need)].ICD_CODE.unique()))

In [ ]:
specific_need = D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE.isin(specific_need)]
specific_need.head()

In [ ]:
matched_9 = list(pd.unique(list(specific_need[specific_need.ICD_VERSION == 9].ICD_CODE.values)))
matched_10 = list(pd.unique(list(specific_need[specific_need.ICD_VERSION == 10].ICD_CODE.values)))
print(len(matched_9),len(matched_10))

In [ ]:
matched_9_isin = only_diag_9_10[only_diag_9_10.ICD_CODE.isin(matched_9)].ICD_CODE.values 
matched_9_notin = list(set(matched_9)-set(matched_9_isin))  
print(len(matched_9_isin),len(matched_9_notin))

matched_10_add = list(pd.unique(list(matched_10) + list(pd.unique(only_diag_9_10[only_diag_9_10.ICD_CODE.isin(matched_9)].ROOT.values))))
print(len(matched_10_add))

In [ ]:
def diag_9_10_check(df,diag_9_10,icds,dataset,n):
    if dataset == 'MIMICIV':
        print(dataset, 'ICD_VERSION:',Counter(df.ICD_VERSION))
    else:
        df['ICD_VERSION'] = 9
        print(dataset,'ICD_VERSION:',Counter(df.ICD_VERSION))

    d_9 = df[df.ICD_VERSION == 9]
    d_10 = df[df.ICD_VERSION == 10]
    print(d_9.shape,d_10.shape)
    
    po_9 = d_9[d_9.ICD_CODE.isin(icds[icds.ICD_VERSION ==9].ICD_CODE)]
    po_10 = d_10[d_10.ICD_CODE.isin(icds[icds.ICD_VERSION ==10].ICD_CODE)]

    all_9code = po_9.ICD_CODE.value_counts()
    all_9code = all_9code[all_9code>n].index
    print(len(all_9code), all_9code)
    all_10code = po_10.ICD_CODE.value_counts()
    all_10code = all_10code[all_10code>n].index
    print(len(all_10code), all_10code)
    
    cc_9code = po_9[po_9.ICD_CODE.isin(diag_9_10[diag_9_10.ICD_CODE.isin(po_9.ICD_CODE.unique())].ICD_CODE)].ICD_CODE.value_counts()
    cc_9code = cc_9code[cc_9code>n].index
    print(len(cc_9code), cc_9code)
    print(len(list(set(all_9code) - set(cc_9code))), list(set(all_9code) - set(cc_9code)))
    cc_9code_10 = list(diag_9_10[diag_9_10.ICD_CODE.isin(cc_9code)].ROOT.unique())
    print(len(cc_9code_10),cc_9code_10)

    cc_10code = po_10[po_10.ICD_CODE.isin(diag_9_10[diag_9_10.ROOT.isin(po_10.ICD_CODE.unique())].ICD_CODE)].ICD_CODE.value_counts()
    cc_10code = cc_10code[cc_10code>n]

    return list(pd.unique(list(all_10code)+list(cc_9code_10))), list(set(all_9code) - set(cc_9code))

In [ ]:
IV_diagnoses_raw = pd.read_csv('/hosp/diagnoses_icd.csv.gz')
IV_diagnoses_raw.columns = IV_diagnoses_raw.columns.str.upper()
IV_diagnoses_raw = IV_diagnoses_raw[['SUBJECT_ID', 'HADM_ID', 'ICD_CODE','ICD_VERSION']]
print(IV_diagnoses_raw.shape, len(IV_diagnoses_raw.HADM_ID.unique()),len(IV_diagnoses_raw.ICD_CODE.unique()))

In [ ]:
D_10code, D_9code = diag_9_10_check(IV_diagnoses_raw,only_diag_9_10,D_ICD_DIAGNOSES,'MIMICIV',3000)

In [ ]:
print(len(D_10code),len(D_9code))

In [ ]:
d_10_iii = ['I95', 'E87', 'R10', 'F10', 'Z21', 'R68', 'J44', 'F30', 'K70', 'J69', 'I50', 'J98', 'N17', 'K92', 'R09', 'I47', 'D64', 'I25', 'M15', 'M81', 'R58', 'F41', 'I10', 'I34', 'I21', 'N18', 'I12', 'E11', 'Z51', 'K22', 'G93', 'G35', 'J45', 'F32', 'M79', 'Z91', 'K59', 'R45', 'I20', 'G43', 'G61', 'E78', 'E66', 'Z95', 'C33', 'J18', 'J41', 'R69', 'R00', 'T82', 'Q20', 'R19', 'K80', 'D50', 'A04', 'F19', 'B37', 'E89', 'N39', 'T78', 'Z00', 'K56', 'K57', 'K29', 'R78', 'D69', 'I42', 'I27', 'G89', 'B15', 'A40', 'T81', 'E83', 'G47', 'D66', 'E44', 'F05', 'K66', 'B95', 'K76', 'K91', 'N28', 'M10', 'L89', 'I44', 'I69', 'J15', 'Z85', 'I24', 'I66', 'G81', 'H40', 'Z81', 'G44', 'K75', 'I85', 'I61', 'K85', 'N23', 'K41', 'D70', 'K31', 'J93', 'N40', 'H35', 'C79', 'I31', 'I33', 'G40', 'M48', 'I73', 'M60', 'F04', 'L03', 'G60', 'S06', 'G30', 'I82', 'S12', 'I26', 'M06', 'E22', 'S27', 'C78', 'G97', 'G20', 'J84', 'I67', 'I60', 'D61', 'I71', 'B20', 'J43', 'Z16', 'I70', 'C22', 'I07', 'R06', 'C61', 'I62', 'N05', 'I08', 'C82', 'N13', 'J95', 'S01', 'M32', 'Z23']
len(d_10_iii)

In [ ]:
d_9_iii = ['E9342', 'V1254', 'V433', 'E8790', 'V4364', 'E8859', 'E9320', '76519', 'V502', '7470', 'V290', '99731', 'V1259', '76527', '7742', '6930', 'E9478', '7756', '76517', 'E8889', 'V3000', '76516', '77081', 'E8490', 'V293', 'V422', '9974', '86121', '7464', '9973', '43310', '7793', 'E8786', 'E8781', 'V3101', 'E8788', '39891', 'E8498', 'E8780', 'V4365', '7746', 'V420', '769', '9971', '76518', 'E8809', '7661', '2113', 'V441', 'V1251', 'V8741', '77089', '5570', 'E8798', 'E9331', '9975', 'E8497', '7706', '99702', '76528', 'V3001', 'V440', 'V462', 'E8782', '76526', '77981', '7766', '77989']
len(d_9_iii)

In [ ]:
d_10_iii_iv = list(pd.unique(d_10_iii + D_10code))
d_9_iii_iv = list(pd.unique(d_9_iii + D_9code))
print(len(d_10_iii_iv),len(d_9_iii_iv))

In [ ]:
del(D_10code, D_9code,d_10_iii,d_9_iii)

In [ ]:
label_9 = PCOS_9 + Neoplasm_ovary_9 + Endometriosis_9 + Leiomyoma_9 + Va_uti_9 + check_AKI_9 + check_CKD_9
label_10 = PCOS_10 + Neoplasm_ovary_10 + Endometriosis_10 + Leiomyoma_10 + Va_uti_10 + check_AKI_10 + check_CKD_10
print(len(label_9), len(label_10))

In [ ]:
Final_9 = list(pd.unique(d_9_iii_iv + matched_9_notin))
Final_9 = list(set(Final_9)-set(['E8809','E8889', 'V270', 'V4364', 'V446', 'V600','E894']))
Final_10 = list(pd.unique(d_10_iii_iv + matched_10_add))
Final_10 = list(set(Final_10)-set(['E8339', 'E8342','N40', 'N400', 'N401', 'Z8546','V433','V462','V474','V536','V556','V290', 'V293', 'V420', 'V422', 'V433','V440', 'V441', 'V462', 'V502']))

print('Final_9：',len(Final_9))
print('Final_10：',len(Final_10))

In [ ]:
len(pd.unique(Final_9+Final_10))

In [ ]:
len(pd.unique(Final_9+Final_10+label_9+label_10))

In [ ]:
def get_diag(df,Final_9,Final_10,label_9,label_10,dataset,patients,only_diag_9_10):
    if dataset == 'MIMICIV':
        print(dataset,'ICD_VERSION:',Counter(df.ICD_VERSION))
    else:
        df['ICD_VERSION'] = 9
        print(dataset,'ICD_VERSION:',Counter(df.ICD_VERSION))
        
    df_diag_9 = df[df.ICD_VERSION == 9]
    df_diag_10 = df[df.ICD_VERSION == 10]
    print('ICD 9:',df_diag_9.shape, 'ICD 10:',df_diag_10.shape)

    df_diag_9_add = df_diag_9[df_diag_9.ICD_CODE.isin(list(pd.unique(Final_9+label_9)))] 
    df_diag_9_mapped = df_diag_9[~(df_diag_9.ICD_CODE.isin(Final_9))] 
    df_diag_9_mapped = pd.merge(df_diag_9_mapped,only_diag_9_10,on='ICD_CODE',how='left')
    df_diag_9_mapped = df_diag_9_mapped[df_diag_9_mapped.ROOT.isin(list(pd.unique(Final_10+label_10)))]

    df_diag_10 = df_diag_10[df_diag_10.ICD_CODE.isin(list(pd.unique(Final_10+label_10)))]
    print('ICD_10:', len(df_diag_10.ICD_CODE.unique()))

    all_df_diag = pd.concat([df_diag_9_mapped,df_diag_9_add])
    all_df_diag = pd.concat([all_df_diag,df_diag_10])
    all_df_diag['ROOT'] = all_df_diag['ROOT'].fillna(all_df_diag['ICD_CODE'])
    all_df_diag = all_df_diag.drop_duplicates(subset=['HADM_ID', 'ICD_CODE', 'ICD_VERSION', 'ROOT'], keep='first')
    print(all_df_diag.shape, len(all_df_diag.HADM_ID.unique()), len(all_df_diag.ICD_CODE.unique()), len(all_df_diag.ROOT.unique()))
    print(len(list(all_df_diag.ROOT.unique())))

    all_df_diag = all_df_diag[all_df_diag.HADM_ID.isin(patients.HADM_ID)]
    print(all_df_diag.shape, len(all_df_diag.HADM_ID.unique()), len(all_df_diag.ICD_CODE.unique()), len(all_df_diag.ROOT.unique()))
    print(len(list(all_df_diag.ROOT.unique())))

    # Group by subject and admission ID to list ICD codes per ROOT category
    all_df_diag = all_df_diag.groupby(['SUBJECT_ID', 'HADM_ID', 'ROOT'])['ICD_CODE'].apply(list).reset_index()
    # Pivot the grouped data to create a matrix for HADM_ID and ROOT categories
    all_df_diag = all_df_diag.pivot(index='HADM_ID', columns='ROOT', values='ICD_CODE')
    all_df_diag = all_df_diag.reset_index()
    print(all_df_diag.shape,len(all_df_diag.HADM_ID.unique()))
    
    all_df_diag.loc[:, all_df_diag.columns != 'HADM_ID'] = all_df_diag.loc[:, all_df_diag.columns != 'HADM_ID'].notna().astype(int)
    zero_cols = all_df_diag.columns[all_df_diag.eq(0).all()].tolist()
    zero_cols = list(set(zero_cols)-set(label_9+label_10))
    all_df_diag = all_df_diag.drop(columns=zero_cols)
    print(all_df_diag.shape,len(all_df_diag.HADM_ID.unique()))

    all_df_diag.columns = [
    ('Diag_9_' + col if col in (Final_9 + label_9) 
     else 'Diag_10_' + col if col in (Final_10 + label_10) 
     else col)
    for col in all_df_diag.columns]

    return all_df_diag

In [ ]:
all_df_diag = get_diag(IV_diagnoses_raw,Final_9,Final_10,label_9,label_10,'MIMICIV',df,only_diag_9_10)

In [ ]:
all_df_diag.head()

In [ ]:
print(len(label_9),len(label_10))
print(len(pd.unique(label_9)),len(pd.unique(label_10)))
now_l_9 = list(pd.unique(label_9))
now_l_9 = ['Diag_9_'+i for i in now_l_9]
now_l_10 = list(pd.unique(label_10))
now_l_10 = ['Diag_10_'+i for i in now_l_10]
print(len(now_l_9),len(now_l_10))

existing_l_9 = list(set(now_l_9) & set(all_df_diag.columns))
existing_l_10 = list(set(now_l_10) & set(all_df_diag.columns))
print(len(existing_l_9),len(existing_l_10))

existing_AKI = list(set(['Diag_9_'+i for i in check_AKI_9]+['Diag_10_'+i for i in check_AKI_10]) & set(all_df_diag.columns))
existing_CKD = list(set(['Diag_9_'+i for i in check_CKD_9]+['Diag_10_'+i for i in check_CKD_10]) & set(all_df_diag.columns))
existing_PCOS = list(set(['Diag_9_'+i for i in PCOS_9]+['Diag_10_'+i for i in PCOS_10]) & set(all_df_diag.columns))
existing_Neoplasm_ovary = list(set(['Diag_9_'+i for i in Neoplasm_ovary_9]+['Diag_10_'+i for i in Neoplasm_ovary_10]) & set(all_df_diag.columns))
existing_Endometriosis = list(set(['Diag_9_'+i for i in Endometriosis_9]+['Diag_10_'+i for i in Endometriosis_10]) & set(all_df_diag.columns))
existing_Leiomyoma = list(set(['Diag_9_'+i for i in Leiomyoma_9]+['Diag_10_'+i for i in Leiomyoma_10]) & set(all_df_diag.columns))
existing_Va_uti = list(set(['Diag_9_'+i for i in Va_uti_9]+['Diag_10_'+i for i in Va_uti_10]) & set(all_df_diag.columns))
print(len(existing_AKI),len(existing_CKD),len(existing_PCOS),len(existing_Neoplasm_ovary),len(existing_Endometriosis),len(existing_Leiomyoma),len(existing_Va_uti))

In [ ]:
all_df_diag['Group_AKI'] = (all_df_diag[existing_AKI] == 1).any(axis=1).astype(int)
print(Counter(all_df_diag['Group_AKI']), Counter(all_df_diag['Group_AKI']))
print('\n')

all_df_diag['Group_CKD'] = (all_df_diag[existing_CKD] == 1).any(axis=1).astype(int)
print(Counter(all_df_diag['Group_CKD']), Counter(all_df_diag['Group_CKD']))
print('\n')

all_df_diag['Group_PCOS'] = (all_df_diag[existing_PCOS] == 1).any(axis=1).astype(int)
print(Counter(all_df_diag['Group_PCOS']))
print(len(existing_PCOS), existing_PCOS)
for c in existing_PCOS:
    print(c,D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE==c.split('_')[-1]].ICD_TEXT.values[0],Counter(all_df_diag[c]))
print('\n')

all_df_diag['Group_Neoplasm_ovary'] = (all_df_diag[existing_Neoplasm_ovary] == 1).any(axis=1).astype(int)
print(Counter(all_df_diag['Group_Neoplasm_ovary']))
print(len(existing_Neoplasm_ovary), existing_Neoplasm_ovary)
for c in existing_Neoplasm_ovary:
    print(c,D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE==c.split('_')[-1]].ICD_TEXT.values[0],Counter(all_df_diag[c]))
print('\n')

all_df_diag['Group_Endometriosis'] = (all_df_diag[existing_Endometriosis] == 1).any(axis=1).astype(int)
print(Counter(all_df_diag['Group_Endometriosis']))
print(len(existing_Endometriosis), existing_Endometriosis)
for c in existing_Endometriosis:
    print(c,D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE==c.split('_')[-1]].ICD_TEXT.values[0],Counter(all_df_diag[c]))
print('\n')

all_df_diag['Group_Leiomyoma'] = (all_df_diag[existing_Leiomyoma] == 1).any(axis=1).astype(int)
print(Counter(all_df_diag['Group_Leiomyoma']))
print(len(existing_Leiomyoma), existing_Leiomyoma)
for c in existing_Leiomyoma:
    print(c,D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE==c.split('_')[-1]].ICD_TEXT.values[0],Counter(all_df_diag[c]))
print('\n')

all_df_diag['Group_Va_uti'] = (all_df_diag[existing_Va_uti] == 1).any(axis=1).astype(int)
print(Counter(all_df_diag['Group_Va_uti']))
print(len(existing_Va_uti), existing_Va_uti)
for c in existing_Va_uti:
    print(c,D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE==c.split('_')[-1]].ICD_TEXT.values[0],Counter(all_df_diag[c]))
print('\n')

In [ ]:
all_df_diag.head()

In [ ]:
all_df_diag.shape

In [ ]:
all_df_diag.to_csv('/IV_all_diag.csv',index=False)

### Check the diagnostic information of III and IV, and combine ICDs with the same meaning

In [ ]:
existing_labels = existing_AKI + existing_CKD
len(existing_labels)

In [ ]:
print(all_df_diag.shape)
all_df_diag = all_df_diag.drop(columns=existing_labels, errors='ignore')
all_df_diag.shape

In [ ]:
iv_ids = [i for i in all_df_diag.columns if 'Diag_' in i]
len(iv_ids)

In [ ]:
iii_ids = ['Diag_9_2113', 'Diag_9_39891', 'Diag_9_43310', 'Diag_9_5570', 'Diag_9_64891', 'Diag_9_66411', 'Diag_9_6930', 'Diag_9_7464', 'Diag_9_7470', 'Diag_9_86121', 'Diag_9_99702', 'Diag_9_9971', 'Diag_9_9973', 'Diag_9_99731', 'Diag_9_9974', 'Diag_9_9975', 'Diag_10_A04', 'Diag_10_A40', 'Diag_10_B15', 'Diag_10_B20', 'Diag_10_B37', 'Diag_10_B95', 'Diag_10_C22', 'Diag_10_C33', 'Diag_10_C61', 'Diag_10_C78', 'Diag_10_C79', 'Diag_10_C82', 'Diag_10_D50', 'Diag_10_D61', 'Diag_10_D64', 'Diag_10_D66', 'Diag_10_D69', 'Diag_10_D70', 'Diag_10_E11', 'Diag_10_E22', 'Diag_10_E28', 'Diag_10_E43', 'Diag_10_E44', 'Diag_10_E66', 'Diag_10_E78', 'Diag_10_E83', 'Diag_9_E8490', 'Diag_9_E8497', 'Diag_9_E8498', 'Diag_9_E8499', 'Diag_10_E87', 'Diag_9_E8780', 'Diag_9_E8781', 'Diag_9_E8782', 'Diag_9_E8786', 'Diag_9_E8788', 'Diag_9_E8790', 'Diag_9_E8798', 'Diag_9_E8859', 'Diag_10_E89', 'Diag_9_E9320', 'Diag_9_E9331', 'Diag_9_E9342', 'Diag_9_E9478', 'Diag_10_F04', 'Diag_10_F05', 'Diag_10_F10', 'Diag_10_F19', 'Diag_10_F30', 'Diag_10_F32', 'Diag_10_F41', 'Diag_10_F43', 'Diag_10_G20', 'Diag_10_G30', 'Diag_10_G35', 'Diag_10_G40', 'Diag_10_G43', 'Diag_10_G44', 'Diag_10_G47', 'Diag_10_G60', 'Diag_10_G61', 'Diag_10_G81', 'Diag_10_G89', 'Diag_10_G93', 'Diag_10_G97', 'Diag_10_H35', 'Diag_10_H40', 'Diag_10_I07', 'Diag_10_I08', 'Diag_10_I10', 'Diag_10_I12', 'Diag_10_I20', 'Diag_10_I21', 'Diag_10_I24', 'Diag_10_I25', 'Diag_10_I26', 'Diag_10_I27', 'Diag_10_I31', 'Diag_10_I33', 'Diag_10_I34', 'Diag_10_I42', 'Diag_10_I44', 'Diag_10_I47', 'Diag_10_I50', 'Diag_10_I60', 'Diag_10_I61', 'Diag_10_I62', 'Diag_10_I66', 'Diag_10_I67', 'Diag_10_I69', 'Diag_10_I70', 'Diag_10_I71', 'Diag_10_I73', 'Diag_10_I82', 'Diag_10_I85', 'Diag_10_I95', 'Diag_10_J15', 'Diag_10_J18', 'Diag_10_J41', 'Diag_10_J43', 'Diag_10_J44', 'Diag_10_J45', 'Diag_10_J69', 'Diag_10_J84', 'Diag_10_J93', 'Diag_10_J95', 'Diag_10_J98', 'Diag_10_K22', 'Diag_10_K29', 'Diag_10_K31', 'Diag_10_K41', 'Diag_10_K56', 'Diag_10_K57', 'Diag_10_K59', 'Diag_10_K66', 'Diag_10_K70', 'Diag_10_K75', 'Diag_10_K76', 'Diag_10_K80', 'Diag_10_K85', 'Diag_10_K91', 'Diag_10_K92', 'Diag_10_L03', 'Diag_10_L89', 'Diag_10_M06', 'Diag_10_M10', 'Diag_10_M15', 'Diag_10_M32', 'Diag_10_M48', 'Diag_10_M60', 'Diag_10_M79', 'Diag_10_M81', 'Diag_10_N05', 'Diag_10_N13', 'Diag_10_N23', 'Diag_10_N28', 'Diag_10_N39', 'Diag_10_N72', 'Diag_10_N92', 'Diag_10_O75', 'Diag_10_Q20', 'Diag_10_R00', 'Diag_10_R06', 'Diag_10_R09', 'Diag_10_R10', 'Diag_10_R19', 'Diag_10_R45', 'Diag_10_R58', 'Diag_10_R63', 'Diag_10_R68', 'Diag_10_R69', 'Diag_10_R78', 'Diag_10_S01', 'Diag_10_S06', 'Diag_10_S12', 'Diag_10_S27', 'Diag_10_T78', 'Diag_10_T81', 'Diag_10_T82', 'Diag_9_V1251', 'Diag_9_V1254', 'Diag_9_V1255', 'Diag_9_V1259', 'Diag_9_V420', 'Diag_9_V422', 'Diag_9_V433', 'Diag_9_V4365', 'Diag_9_V440', 'Diag_9_V441', 'Diag_9_V462', 'Diag_9_V6284', 'Diag_9_V8741', 'Diag_10_Z00', 'Diag_10_Z16', 'Diag_10_Z21', 'Diag_10_Z23', 'Diag_10_Z51', 'Diag_10_Z80', 'Diag_10_Z81', 'Diag_10_Z85', 'Diag_10_Z87', 'Diag_10_Z91', 'Diag_10_Z95']
iv_ids = list(pd.unique(iv_ids+iii_ids))
len(iv_ids)

In [ ]:
iv_ids_dict = pd.DataFrame()
for i in iv_ids:
    iv_ids_dict = pd.concat([iv_ids_dict, D_ICD_DIAGNOSES[(D_ICD_DIAGNOSES.ICD_CODE == i.split('_')[-1])&(D_ICD_DIAGNOSES.ICD_VERSION == int(i.split('_')[1]))]])
iv_ids_dict = iv_ids_dict.sort_values(by=['ICD_TEXT'])
duplicated = iv_ids_dict[iv_ids_dict.duplicated(subset=['ICD_TEXT'],keep=False)]

In [ ]:
dupi_ids = []
for i in pd.unique(duplicated['ICD_TEXT'].values):
    print('\n',i)
    
    this_rows = duplicated[duplicated['ICD_TEXT'] == i]
    
    this_cols = ('Diag_' + this_rows['ICD_VERSION'].astype(str) + '_' + this_rows['ICD_CODE'].astype(str)).tolist()
    dupi_ids.extend(this_cols)
    
    this_cols_existing = [col for col in this_cols if col in all_df_diag.columns]
    
    for j in this_cols:
        if j in all_df_diag.columns:
            print(Counter(all_df_diag[j]))
        else:
            print(f"  Column {j} not found in all_df_diag for {i}")

    if not this_cols_existing:
        print(f"  No matching columns in all_df_diag for {i}")
        continue

    all_df_diag['Diag_' + i] = all_df_diag[this_cols_existing].eq(1).any(axis=1).astype(int)
    
    print(Counter(all_df_diag['Diag_' + i]))


In [ ]:
print(all_df_diag.shape)
all_df_diag = all_df_diag.drop(columns=dupi_ids, errors='ignore')
all_df_diag.shape

In [ ]:
all_df_diag.shape

In [ ]:
f_iii = ['HADM_ID', 'Diag_9_2113', 'Diag_9_39891', 'Diag_9_43310', 'Diag_9_5570', 'Diag_9_64891', 'Diag_9_66411', 'Diag_9_6930', 'Diag_9_7464', 'Diag_9_7470', 'Diag_9_86121', 'Diag_9_99702', 'Diag_9_9971', 'Diag_9_9973', 'Diag_9_99731', 'Diag_9_9974', 'Diag_9_9975', 'Diag_10_A04', 'Diag_10_A40', 'Diag_10_B15', 'Diag_10_B20', 'Diag_10_B37', 'Diag_10_B95', 'Diag_10_C22', 'Diag_10_C33', 'Diag_10_C61', 'Diag_10_C78', 'Diag_10_C79', 'Diag_10_C82', 'Diag_10_D50', 'Diag_10_D61', 'Diag_10_D64', 'Diag_10_D66', 'Diag_10_D69', 'Diag_10_D70', 'Diag_10_E11', 'Diag_10_E22', 'Diag_10_E28', 'Diag_10_E43', 'Diag_10_E44', 'Diag_10_E66', 'Diag_10_E78', 'Diag_10_E83', 'Diag_9_E8490', 'Diag_9_E8497', 'Diag_9_E8498', 'Diag_9_E8499', 'Diag_10_E87', 'Diag_9_E8780', 'Diag_9_E8781', 'Diag_9_E8782', 'Diag_9_E8786', 'Diag_9_E8788', 'Diag_9_E8790', 'Diag_9_E8798', 'Diag_9_E8859', 'Diag_10_E89', 'Diag_9_E9320', 'Diag_9_E9331', 'Diag_9_E9342', 'Diag_9_E9478', 'Diag_10_F04', 'Diag_10_F05', 'Diag_10_F10', 'Diag_10_F19', 'Diag_10_F30', 'Diag_10_F32', 'Diag_10_F41', 'Diag_10_F43', 'Diag_10_G20', 'Diag_10_G30', 'Diag_10_G35', 'Diag_10_G40', 'Diag_10_G43', 'Diag_10_G44', 'Diag_10_G47', 'Diag_10_G60', 'Diag_10_G61', 'Diag_10_G81', 'Diag_10_G89', 'Diag_10_G93', 'Diag_10_G97', 'Diag_10_H35', 'Diag_10_H40', 'Diag_10_I07', 'Diag_10_I08', 'Diag_10_I10', 'Diag_10_I12', 'Diag_10_I20', 'Diag_10_I21', 'Diag_10_I24', 'Diag_10_I25', 'Diag_10_I26', 'Diag_10_I27', 'Diag_10_I31', 'Diag_10_I33', 'Diag_10_I34', 'Diag_10_I42', 'Diag_10_I44', 'Diag_10_I47', 'Diag_10_I50', 'Diag_10_I60', 'Diag_10_I61', 'Diag_10_I62', 'Diag_10_I66', 'Diag_10_I67', 'Diag_10_I69', 'Diag_10_I70', 'Diag_10_I71', 'Diag_10_I73', 'Diag_10_I82', 'Diag_10_I85', 'Diag_10_I95', 'Diag_10_J15', 'Diag_10_J41', 'Diag_10_J43', 'Diag_10_J44', 'Diag_10_J45', 'Diag_10_J69', 'Diag_10_J84', 'Diag_10_J93', 'Diag_10_J95', 'Diag_10_J98', 'Diag_10_K22', 'Diag_10_K29', 'Diag_10_K31', 'Diag_10_K41', 'Diag_10_K56', 'Diag_10_K57', 'Diag_10_K59', 'Diag_10_K66', 'Diag_10_K70', 'Diag_10_K75', 'Diag_10_K76', 'Diag_10_K80', 'Diag_10_K85', 'Diag_10_K91', 'Diag_10_K92', 'Diag_10_L03', 'Diag_10_L89', 'Diag_10_M06', 'Diag_10_M10', 'Diag_10_M15', 'Diag_10_M32', 'Diag_10_M48', 'Diag_10_M60', 'Diag_10_M79', 'Diag_10_M81', 'Diag_10_N05', 'Diag_10_N13', 'Diag_10_N23', 'Diag_10_N28', 'Diag_10_N39', 'Diag_10_N72', 'Diag_10_N92', 'Diag_10_O75', 'Diag_10_Q20', 'Diag_10_R00', 'Diag_10_R06', 'Diag_10_R09', 'Diag_10_R10', 'Diag_10_R19', 'Diag_10_R45', 'Diag_10_R58', 'Diag_10_R63', 'Diag_10_R68', 'Diag_10_R69', 'Diag_10_R78', 'Diag_10_S01', 'Diag_10_S06', 'Diag_10_S12', 'Diag_10_S27', 'Diag_10_T78', 'Diag_10_T81', 'Diag_10_T82', 'Diag_9_V1251', 'Diag_9_V1259', 'Diag_9_V420', 'Diag_9_V422', 'Diag_9_V433', 'Diag_9_V4365', 'Diag_9_V440', 'Diag_9_V441', 'Diag_9_V462', 'Diag_9_V6284', 'Diag_10_Z00', 'Diag_10_Z16', 'Diag_10_Z21', 'Diag_10_Z23', 'Diag_10_Z51', 'Diag_10_Z80', 'Diag_10_Z81', 'Diag_10_Z85', 'Diag_10_Z87', 'Diag_10_Z91', 'Diag_10_Z95', 'Group_AKI', 'Group_CKD', 'Group_PCOS', 'Group_Neoplasm_ovary', 'Group_Endometriosis', 'Group_Leiomyoma', 'Group_Va_uti', 'Diag_Personal history of antineoplastic chemotherapy', 'Diag_Personal history of pulmonary embolism', 'Diag_Personal history of transient ischemic attack (TIA), and cerebral infarction without residual deficits', 'Diag_Pneumonia, unspecified organism']
len(f_iii)

In [ ]:
print(list(set(f_iii)-set(all_df_diag.columns)))

In [ ]:
oii = list(set(all_df_diag.columns)-set(f_iii+['Diag_10_A419','Diag_10_D62','Diag_10_R6521','Diag_10_Z794']))
oii_id = [i.split('_')[-1] for i in oii if 'Diag_' in i]
D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE.isin(oii_id)]#.ICD_TEXT.values

In [ ]:
print(oii)

In [ ]:
print(all_df_diag.shape)
all_df_diag = all_df_diag.drop(columns=oii, errors='ignore')
all_df_diag.shape

In [ ]:
oii = list(set(all_df_diag.columns)-set(f_iii+['Diag_10_A419','Diag_10_D62','Diag_10_R6521','Diag_10_Z794']))
oii_id = [i.split('_')[-1] for i in oii if 'Diag_' in i]
D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE.isin(oii_id)]#.ICD_TEXT.values

In [ ]:
all_df_diag.to_csv('/IV_all_diag_matchIII.csv',index=False)

In [ ]:
print(all_df_diag.columns.tolist())

# Procedure

In [ ]:
df = pd.read_csv('/MIMIC4_p.csv')
print('------------------------------------------------------------------------')
print('Final Data：',df.shape, 'SUBJECT_ID:',len(df.SUBJECT_ID.unique()),'HADM_ID:',len(df.HADM_ID.unique()), 'ICUSTAY_ID:',len(df.ICUSTAY_ID.unique()))
print('DIEINHOSPITAL:',Counter(df['DIEINHOSPITAL']))
print('DIEINICU:',Counter(df['DIEINICU']))
print('Readmission_30:',Counter(df['Readmission_30']))
print('Readmission_60:',Counter(df['Readmission_60']))
print('ICU_within_12hr_of_admit:',Counter(df['ICU_within_12hr_of_admit']))
print('Multiple_ICUs:',Counter(df['Multiple_ICUs']))

In [ ]:
d_iii_icd_procedures = pd.read_csv('/mimic-iii-clinical-database-1.4/d_icd_procedures.csv.gz')
d_iii_icd_procedures = d_iii_icd_procedures.rename(columns={'ICD9_CODE':'ICD_CODE'})
d_iii_icd_procedures = d_iii_icd_procedures.rename(columns={'LONG_TITLE':'ICD_TEXT'})
d_iii_icd_procedures['ICD_VERSION'] = 9
d_iii_icd_procedures = d_iii_icd_procedures[['ICD_CODE', 'ICD_VERSION', 'ICD_TEXT']]
d_iii_icd_procedures['ICD_CODE'] = d_iii_icd_procedures['ICD_CODE'].astype(str)

d_iv_icd_procedures = pd.read_csv('/3.1/hosp/d_icd_procedures.csv.gz')
d_iv_icd_procedures.columns = ['ICD_CODE', 'ICD_VERSION', 'ICD_TEXT']
d_iv_icd_procedures['ICD_CODE'] = d_iv_icd_procedures['ICD_CODE'].astype(str)

d_icd_procedures = pd.concat([d_iii_icd_procedures,d_iv_icd_procedures])
d_icd_procedures = d_icd_procedures.drop_duplicates(keep='first')

dd = d_icd_procedures[d_icd_procedures.duplicated(subset=['ICD_TEXT'],keep=False)].sort_values(by='ICD_TEXT')
dd = (
    dd.groupby('ICD_TEXT')
    .agg(
        final_ICD_CODE=('ICD_CODE', lambda x: max(x, key=len)),  
        all_ICD_CODEs=('ICD_CODE', lambda x: list(x))           
    )
    .reset_index()
)

d_icd_procedures = (
    d_icd_procedures.assign(code_len=d_icd_procedures['ICD_CODE'].str.len())
    .sort_values('code_len', ascending=False)
    .drop_duplicates(subset=['ICD_TEXT'], keep='first')
    .drop(columns='code_len')
)

p_map = {
    code: final 
    for final, codes in zip(dd['final_ICD_CODE'], dd['all_ICD_CODEs']) 
    for code in codes if code!=final
}

print(d_icd_procedures.ICD_VERSION.value_counts())
d_icd_procedures.head(2)

In [ ]:
print(d_icd_procedures.shape,len(d_icd_procedures.ICD_CODE.unique()))
d_icd_procedures[d_icd_procedures.duplicated('ICD_CODE',keep=False)]#.ICD_TEXT.values

In [ ]:
p_raw = pd.read_csv('/hosp/procedures_icd.csv.gz')
p_raw.columns = p_raw.columns.str.upper()
p_raw = p_raw[['SUBJECT_ID', 'HADM_ID','ICD_CODE']]
p_raw['ICD_CODE'] = p_raw['ICD_CODE'].astype(str)
p_raw['ICD_CODE'] = p_raw['ICD_CODE'].replace(p_map)
p_raw = p_raw.drop_duplicates(keep='first')
print('IV_p_raw',p_raw.shape,len(p_raw.HADM_ID.unique()))

In [ ]:
p_raw.head(2)

In [ ]:
tt = p_raw.ICD_CODE.value_counts()
most_p_ids = list(tt[tt>2000].index)
print(len(most_p_ids))

In [ ]:
iii_p = ['3893', '9604', '966', '9671', '9904', '3961', '9672', '9955', '8856', '3891', '3615', '9915', '8872', '3722', '3995', '9390', '3324', '3723', '4513', '9983', '640', '8853', '0331', '9907', '3612', '311', '9920', '3491', '3404', '0040', '4311', '3895', '3613', '3897', '5491', '3521', '8841', '0066', '9905', '3606', '3607', '8964', '3601', '3322', '3323', '3761', '3611', '0045', '0017', '8855', '9962', '4523', '387', '8842', '0014', '8659', '3950', '5459', '9910', '8847', '0159', '9960', '4443', '4516', '9605', '8848', '3721', '9607', '8622', '8852', '3845', '3522', '9925']
len(iii_p)

In [ ]:
#included Procedures id
icds = [9604, 3722, 3723, 225966, 8856, 224270, 3995, 224385, 3615, 225402, 4513, 8872, 9390, 9904, 3891, 3893, 225401, 3897, 9915, 224264, 966, 9671, 9672, 221217, 3961, 221214, 225792, 225454, 225752, 227194, 3324]
icds = [str(i) for i in icds]
icds = list(pd.unique(icds + most_p_ids+iii_p))
len(icds)

In [ ]:
icds = list(d_icd_procedures[d_icd_procedures.ICD_CODE.isin(icds)].ICD_CODE.values)
len(icds)

In [ ]:
print(icds)

In [ ]:
p_raw = p_raw[p_raw.ICD_CODE.isin(icds)][['HADM_ID','ICD_CODE']].pivot_table(index='HADM_ID', columns='ICD_CODE', aggfunc='size', fill_value=np.nan)
p_raw = p_raw.reset_index()

p_raw.loc[:, p_raw.columns != 'HADM_ID'] = p_raw.loc[:, p_raw.columns != 'HADM_ID'].notna().astype(int)
print('p_raw_result',p_raw.shape,len(p_raw.HADM_ID.unique()))

p_raw = p_raw[p_raw.HADM_ID.isin(df.HADM_ID)]
print('p_raw_result',p_raw.shape,len(p_raw.HADM_ID.unique()))

final_p = df[['HADM_ID','ICUSTAY_ID']]
final_p = pd.merge(final_p,p_raw,on='HADM_ID',how='left')
print(final_p.shape,len(final_p.HADM_ID.unique()),len(final_p.ICUSTAY_ID.unique()))

final_p = final_p[~final_p[final_p.columns.difference(['HADM_ID', 'ICUSTAY_ID'])].isna().all(axis=1)]
final_p = final_p.fillna(0)
print(final_p.shape,len(final_p.HADM_ID.unique()),len(final_p.ICUSTAY_ID.unique()))

final_p.columns = final_p.columns.astype(str)
final_p.columns = ['Pro_' + col if (col != 'HADM_ID')&(col != 'ICUSTAY_ID') else col for col in final_p.columns]
final_p = final_p.loc[:, (final_p != 0).any(axis=0)]
final_p = final_p.drop('HADM_ID', axis=1)
final_p.head()

In [ ]:
zero_cols = final_p.columns[final_p.eq(0).all()].tolist()
zero_cols

In [ ]:
final_p.shape

In [ ]:
final_p.to_csv('/IV_all_pro.csv',index=False)

In [ ]:
iii_cols = ['ICUSTAY_ID', 'Pro_0014', 'Pro_0017', 'Pro_0040', 'Pro_0045',
       'Pro_0066', 'Pro_0159', 'Pro_0331', 'Pro_0741', 'Pro_311', 'Pro_3322',
       'Pro_3323', 'Pro_3324', 'Pro_3404', 'Pro_3491', 'Pro_3521', 'Pro_3522',
       'Pro_3606', 'Pro_3607', 'Pro_3611', 'Pro_3612', 'Pro_3613', 'Pro_3615',
       'Pro_3721', 'Pro_3722', 'Pro_3723', 'Pro_3761', 'Pro_3845', 'Pro_387',
       'Pro_3891', 'Pro_3893', 'Pro_3895', 'Pro_3897', 'Pro_3899', 'Pro_3950',
       'Pro_3961', 'Pro_3995', 'Pro_4311', 'Pro_4443', 'Pro_4513', 'Pro_4516',
       'Pro_4523', 'Pro_5185', 'Pro_5459', 'Pro_5491', 'Pro_7309', 'Pro_734',
       'Pro_7359', 'Pro_7569', 'Pro_8051', 'Pro_8162', 'Pro_8622', 'Pro_8659',
       'Pro_8744', 'Pro_8841', 'Pro_8842', 'Pro_8847', 'Pro_8848', 'Pro_8852',
       'Pro_8853', 'Pro_8855', 'Pro_8856', 'Pro_8872', 'Pro_8938', 'Pro_8964',
       'Pro_9390', 'Pro_9604', 'Pro_9605', 'Pro_9607', 'Pro_966', 'Pro_9671',
       'Pro_9672', 'Pro_9904', 'Pro_9905', 'Pro_9907', 'Pro_9910', 'Pro_9915',
       'Pro_9920', 'Pro_9925', 'Pro_9955', 'Pro_9960', 'Pro_9962']
len(iii_cols)

In [ ]:
print(set(iii_cols)-set(final_p.columns))

In [ ]:
d_icd_procedures[d_icd_procedures.ICD_CODE.isin([i.split('_')[-1] for i in list(set(iii_cols)-set(final_p.columns))])]

In [ ]:
print(set(final_p.columns)-set(iii_cols))

In [ ]:
d_icd_procedures[d_icd_procedures.ICD_CODE.isin([i.split('_')[-1] for i in list(set(final_p.columns)-set(iii_cols))])].ICD_TEXT.values

In [ ]:
final_p.shape

In [ ]:
final_p[iii_cols].to_csv('/IV_all_pro_matchIII.csv',index=False)

# Med

In [ ]:
PRESCRIPTIONS = pd.read_csv('/hosp/prescriptions.csv.gz', dtype={'ndc':'category'})
PRESCRIPTIONS.columns = PRESCRIPTIONS.columns.str.upper()

PRESCRIPTIONS.rename(columns={'STAY_ID': 'ICUSTAY_ID','STARTTIME':'STARTDATE','STOPTIME':'ENDDATE'}, inplace=True)
print(PRESCRIPTIONS.shape)
PRESCRIPTIONS = PRESCRIPTIONS[PRESCRIPTIONS.HADM_ID.isin(df.HADM_ID)]
print(PRESCRIPTIONS.shape)

PRESCRIPTIONS.STARTDATE = pd.to_datetime(PRESCRIPTIONS.STARTDATE)
PRESCRIPTIONS.ENDDATE = pd.to_datetime(PRESCRIPTIONS.ENDDATE)

In [ ]:
PRESCRIPTIONS['DRUG'] = PRESCRIPTIONS['DRUG'].str.strip().replace({

    'Azithromycin ': 'Azithromycin',
    'CeftriaXONE': 'Ceftriaxone',  
    'CefTRIAXone': 'Ceftriaxone',  
    'MetRONIDAZOLE (FLagyl)': 'Metronidazole',
    'Piperacillin-Tazobactam Na': 'Piperacillin-Tazobactam',
    'Piperacillin Tazobactam': 'Piperacillin-Tazobactam',
    'LevETIRAcetam': 'Levetiracetam',
    'CefTAZidime': 'Ceftazidime',
    'CefePIME': 'Cefepime',
    'FoLIC Acid': 'Folic Acid',
    'PredniSONE': 'Prednisone',
    
    'LORazepam': 'Lorazepam',
    'Midazolam HCl': 'Midazolam',
    'HYDROmorphone (Dilaudid)': 'Hydromorphone',
    'OxyCODONE (Immediate Release)': 'Oxycodone',
    'OxycoDONE (Immediate Release) ': 'Oxycodone', 
    'Oxycodone (Immediate Release)': 'Oxycodone',
    'LevETIRAcetam':'Levetiracetam',
    
    'Amiodarone HCl': 'Amiodarone',
    'Metoprolol Tartrate': 'Metoprolol',
    'Metoprolol Succinate XL': 'Metoprolol',
    'Phenylephrine HCl': 'Phenylephrine',
    'Hydralazine Hcl': 'Hydralazine',
    'Ciprofloxacin Hcl':'Ciprofloxacin',
    
    'Heparin Sodium': 'Heparin',
    'Heparin Flush (10 units/ml)': 'Heparin',
    'Heparin Dwell (1000 Units/mL)': 'Heparin',
    'Heparin Flush Cvl  (100 Units/Ml)': 'Heparin',
    
    'Potassium Chloride Replacement (Critical Care and Oncology)': 'Potassium Chloride',
    'Potassium Chloride (Powder)': 'Potassium Chloride',
    'Pantoprazole Sodium': 'Pantoprazole',
    'Aspirin EC': 'Aspirin',
    'Vancomycin HCl': 'Vancomycin',
    'Acetaminophen IV': 'Acetaminophen'
}).str.title()  

In [ ]:
keep_drug = list(pd.unique(['Insulin', 'Furosemide','Magnesium Sulfate','Calcium Gluconate',
'Heparin', 'Metoprolol',
'Acetaminophen',
'Hydromorphone','Fentanyl Citrate','Morphine Sulfate','Oxycodone',
'Propofol','Midazolam','Lorazepam','Dexmedetomidine',
'Aspirin',
'Pantoprazole','Famotidine','Ranitidine','Omeprazole',
'Ondansetron','Metoclopramide',
'Amiodarone','Albumin 25% (12.5g / 50mL)',
'Potassium Chloride',
'Levetiracetam','Gabapentin',
'Meropenem',
'Prednisone','Dexamethasone',
'Tacrolimus','Diltiazem','Thiamine','Folic Acid','Sodium Bicarbonate',
'Vancomycin', 
'Piperacillin-Tazobactam', 
'Meropenem', 
'Cefepime', 
'Ceftazidime',
'Ampicillin-Sulbactam', 
'Linezolid', 
'Azithromycin', 
'Ceftriaxone', 
'Clindamycin',
'Phenylephrine','Norepinephrine','Nitroglycerin', 'Metronidazole',
'Levofloxacin', 'Ciprofloxacin', 'Haloperidol','Simvastatin',
'Atorvastatin','Glucagon','Warfarin','Lactulose']))
len(keep_drug)

In [ ]:
print(keep_drug)

In [ ]:
PRESCRIPTIONS = PRESCRIPTIONS[PRESCRIPTIONS.DRUG.isin(keep_drug)]
print(PRESCRIPTIONS.shape,len(PRESCRIPTIONS.DRUG.unique()))

In [ ]:
iii_m = ['Tacrolimus', 'Warfarin', 'Heparin', 'Furosemide', 'Nitroglycerin', 'Insulin', 'Midazolam', 'Lorazepam', 'Magnesium Sulfate', 'Potassium Chloride', 'Vancomycin', 'Acetaminophen', 'Metoclopramide', 'Morphine Sulfate', 'Calcium Gluconate', 'Pantoprazole', 'Levofloxacin', 'Prednisone', 'Clindamycin', 'Ranitidine', 'Hydromorphone', 'Metoprolol', 'Metronidazole', 'Fentanyl Citrate', 'Haloperidol', 'Piperacillin-Tazobactam', 'Atorvastatin', 'Aspirin', 'Amiodarone', 'Meropenem', 'Phenylephrine', 'Sodium Bicarbonate', 'Propofol', 'Norepinephrine', 'Dexamethasone', 'Ondansetron', 'Levetiracetam', 'Ceftazidime', 'Folic Acid', 'Famotidine', 'Simvastatin', 'Linezolid', 'Ceftriaxone', 'Lactulose', 'Omeprazole', 'Diltiazem', 'Cefepime', 'Oxycodone', 'Azithromycin', 'Gabapentin', 'Ampicillin-Sulbactam', 'Ciprofloxacin', 'Glucagon', 'Dexmedetomidine', 'Thiamine']

In [ ]:
print(set(PRESCRIPTIONS.DRUG.unique())-set(iii_m))

In [ ]:
PRESCRIPTIONS['DOSE_VAL_RX'] = PRESCRIPTIONS['DOSE_VAL_RX'].astype(str).str.replace(',', '', regex=False)
print(PRESCRIPTIONS.shape, PRESCRIPTIONS.shape)

In [ ]:
PRESCRIPTIONS['DOSE_UNIT_RX'] = PRESCRIPTIONS['DOSE_UNIT_RX'].replace({

    'gm': 'g',
    'mg/ml': 'mg/mL',
    'mg/ml': 'mg/mL',
    'mcg/ml': 'mcg/mL',
    'mg/50 ml': 'mg/mL',
    'mg/250 ml': 'mg/mL',
    'mg/m2': 'mg',
    'mg/kg/hr': 'mg',
    'mg/hr': 'mg',
    'mg/min': 'mg',
    'mg/dose': 'mg',
    'mg\\ 0 mg': 'mg',


    'ml': 'mL',
    'L': 'L',


    'Units': 'UNIT',
    'units': 'UNIT',
    '_UNIT': 'UNIT',
    'Units/Liter': 'UNIT',
    'units/hr': 'UNIT',
    'UNIT/HR': 'UNIT',


    'MD to order daily dose': 'dose',
    'Dosing by Pharmacy': 'dose',
    'dose': 'dose',
    'TAB': 'tab',
    'CAP': 'cap',
    'SYR': 'syr',
    'DROP': 'drop',
    'VIAL': 'vial',
    'AMP': 'amp',
    'INJ': 'inj',
    'Appl': 'appl',


    'mcg/kg/min': 'mcg',
    'mcg/kg/hr': 'mcg',
    'mcg/hr': 'mcg',


    'Pick Option Below': None,
    'in': None,  
    np.nan: None,
})


In [ ]:
PRESCRIPTIONS['DOSE_UNIT_RX'].unique()

In [ ]:
PRESCRIPTIONS.DOSE_UNIT_RX.value_counts()

In [ ]:
def process_dose_values(df, column='DOSE_VAL_RX'):

    def parse_dose(value):
        if isinstance(value, str):

            value = value.replace('–', '-').replace('—', '-').replace('－', '-').strip()

            match = re.match(r'^(\d*\.?\d+)\s*-\s*(\d*\.?\d+)$', value)
            if match:
                low = float(match.group(1))
                high = float(match.group(2))
                return (low + high) / 2

        try:
            return float(value)
        except:
            return np.nan

    df[column + '_processed'] = df[column].apply(parse_dose)
    return df

In [ ]:
PRESCRIPTIONS = process_dose_values(PRESCRIPTIONS)

In [ ]:
PRESCRIPTIONS.head(10)[['DOSE_VAL_RX','DOSE_VAL_RX_processed']].head()

In [ ]:
def unify_units(df):
    def convert_group(group):
        if 'DOSE_UNIT_RX' not in group.columns or 'DOSE_VAL_RX_processed' not in group.columns:
            return group
        
        unit_counts = group['DOSE_UNIT_RX'].value_counts(dropna=True)
        if unit_counts.empty:
            return group
        
        target_unit = unit_counts.idxmax()

        conversion_factors = {
            ('mg', 'g'): 1 / 1000,
            ('g', 'mg'): 1000,
            ('mcg', 'mg'): 1 / 1000,
            ('mg', 'mcg'): 1000,
            ('mcg', 'g'): 1 / 1_000_000,
            ('g', 'mcg'): 1_000_000,
        }

        for from_unit in group['DOSE_UNIT_RX'].dropna().unique():
            if from_unit == target_unit:
                continue
            key = (from_unit, target_unit)
            if key in conversion_factors:
                factor = conversion_factors[key]
                mask = group['DOSE_UNIT_RX'] == from_unit
                group.loc[mask, 'DOSE_VAL_RX_processed'] = (
                    group.loc[mask, 'DOSE_VAL_RX_processed'].astype(float) * factor
                )
                group.loc[mask, 'DOSE_UNIT_RX'] = target_unit
            else:
                print(f"⚠️ cant {from_unit} to {target_unit}，keep")

        return group

    # 防止 SettingWithCopyWarning
    df = df.groupby('DRUG', group_keys=False).apply(lambda x: convert_group(x.copy()))
    return df


In [ ]:
PRESCRIPTIONS = unify_units(PRESCRIPTIONS)

In [ ]:
for i in keep_drug:
    print(i,'\n',PRESCRIPTIONS[PRESCRIPTIONS.DRUG == i].DOSE_UNIT_RX.value_counts(),'\n')

In [ ]:
PRESCRIPTIONS_keep_f = pd.merge(PRESCRIPTIONS[['HADM_ID', 'STARTDATE', 'ENDDATE', 'DRUG_TYPE', 'DRUG',
       'GSN', 'NDC', 'DOSE_VAL_RX_processed']],df[['HADM_ID','ICUSTAY_ID','ADMITTIME','DISCHTIME','INTIME','OUTTIME']],on='HADM_ID',how='left')

In [ ]:
df.shape

In [ ]:
PRESCRIPTIONS_keep_f.head(2)

In [ ]:
PRESCRIPTIONS_keep_f.shape

In [ ]:
PRESCRIPTIONS_keep_f.to_csv('/IV_all_med.csv',index=False)

In [ ]:
PRESCRIPTIONS_keep_f.columns

In [ ]:
date_columns = ['ADMITTIME', 'DISCHTIME', 'INTIME', 'OUTTIME','STARTDATE', 'ENDDATE']
PRESCRIPTIONS_keep_f[date_columns] = PRESCRIPTIONS_keep_f[date_columns].apply(pd.to_datetime)

In [ ]:
mask = (PRESCRIPTIONS_keep_f['STARTDATE'] >= PRESCRIPTIONS_keep_f['ADMITTIME']) & (PRESCRIPTIONS_keep_f['ENDDATE'] <= PRESCRIPTIONS_keep_f['OUTTIME'])
PRESCRIPTIONS_keep_f = PRESCRIPTIONS_keep_f[mask]
PRESCRIPTIONS_keep_f.shape

In [ ]:
PRESCRIPTIONS_keep_f['DRUG_hr_inICu'] = PRESCRIPTIONS_keep_f['STARTDATE'] - PRESCRIPTIONS_keep_f['INTIME']
PRESCRIPTIONS_keep_f['DRUG_hr_inICu'] = PRESCRIPTIONS_keep_f['DRUG_hr_inICu'].dt.total_seconds() / 3600

In [ ]:
PRESCRIPTIONS_keep_f.head(2)

In [ ]:
PRESCRIPTIONS_keep_f_48 = PRESCRIPTIONS_keep_f[PRESCRIPTIONS_keep_f.DRUG_hr_inICu<=48]
print(PRESCRIPTIONS_keep_f_48.shape, len(PRESCRIPTIONS_keep_f_48.ICUSTAY_ID.unique()))
PRESCRIPTIONS_keep_f_48 = PRESCRIPTIONS_keep_f_48.groupby(['ICUSTAY_ID', 'DRUG'])['DOSE_VAL_RX_processed'].sum().unstack(fill_value=0).reset_index()
PRESCRIPTIONS_keep_f_48.head()

In [ ]:
PRESCRIPTIONS_keep_f_48.shape

In [ ]:
zero_columns = PRESCRIPTIONS_keep_f_48.columns[(PRESCRIPTIONS_keep_f_48 == 0).all()]
print(list(zero_columns))

In [ ]:
PRESCRIPTIONS_keep_f_48 = PRESCRIPTIONS_keep_f_48.rename(columns={col: f"Med_{col}" for col in PRESCRIPTIONS_keep_f_48.columns if col != 'ICUSTAY_ID'})

In [ ]:
PRESCRIPTIONS_keep_f_48.columns

In [ ]:
nonzero_ratio = (PRESCRIPTIONS_keep_f_48 != 0).sum() / len(PRESCRIPTIONS_keep_f_48)
print(nonzero_ratio)

In [ ]:
PRESCRIPTIONS_keep_f_48.shape

In [ ]:
PRESCRIPTIONS_keep_f_48.to_csv('/IV_all_med_matchIII_48.csv',index=False)

# Static Concat

In [ ]:
df = pd.read_csv('/MIMIC4_p.csv')
print('------------------------------------------------------------------------')
print('Final Data：',df.shape, 'SUBJECT_ID:',len(df.SUBJECT_ID.unique()),'HADM_ID:',len(df.HADM_ID.unique()), 'ICUSTAY_ID:',len(df.ICUSTAY_ID.unique()))
print('DIEINHOSPITAL:',Counter(df['DIEINHOSPITAL']))
print('DIEINICU:',Counter(df['DIEINICU']))
print('Readmission_30:',Counter(df['Readmission_30']))
print('Readmission_60:',Counter(df['Readmission_60']))
print('ICU_within_12hr_of_admit:',Counter(df['ICU_within_12hr_of_admit']))
print('Multiple_ICUs:',Counter(df['Multiple_ICUs']))

In [ ]:
diag = pd.read_csv('/IV_all_diag_matchIII.csv')
proc = pd.read_csv('/IV_all_pro_matchIII.csv')
med = pd.read_csv('/IV_all_med_matchIII_48.csv')

In [ ]:
print(diag.shape, proc.shape, med.shape)

In [ ]:
diag.columns

In [ ]:
proc.columns

In [ ]:
med.columns

In [ ]:
Final_df = pd.merge(df,diag,on='HADM_ID',how='left')
Final_df = pd.merge(Final_df,proc,on='ICUSTAY_ID',how='left')
Final_df = pd.merge(Final_df,med,on='ICUSTAY_ID',how='left')
print(df.shape, Final_df.shape)

In [ ]:
dpm_cols = list(pd.unique(diag.columns.tolist() + proc.columns.tolist() + med.columns.tolist()))
dpm_cols = list(set(dpm_cols)-set(['HADM_ID','ICUSTAY_ID']))
len(dpm_cols)

In [ ]:
iii_acols = ['Pro_9607', 'Diag_9_E8798', 'Diag_9_66411', 'Diag_10_K22', 'Diag_10_R00', 'Pro_3722', 'Diag_10_C33', 'Pro_9905', 'Pro_8841', 'Diag_10_I73', 'Pro_3606', 'Pro_8842', 'Diag_10_I47', 'Diag_9_V420', 'Diag_9_86121', 'Pro_0017', 'Diag_9_99731', 'Diag_10_R09', 'Med_Piperacillin-Tazobactam', 'Pro_3895', 'Pro_8852', 'Diag_10_I67', 'Diag_9_43310', 'Group_Va_uti', 'Diag_9_6930', 'Diag_10_I82', 'Diag_10_R06', 'Med_Potassium Chloride', 'Diag_10_G97', 'Pro_3615', 'Pro_0040', 'Pro_9907', 'Med_Diltiazem', 'Pro_0014', 'Med_Ciprofloxacin', 'Diag_10_B95', 'Diag_10_C61', 'Pro_734', 'Med_Morphine Sulfate', 'Med_Vancomycin', 'Diag_10_I34', 'Diag_10_Z80', 'Diag_10_B37', 'Diag_10_L89', 'Pro_9925', 'Pro_9960', 'Diag_10_D69', 'Diag_9_E9320', 'Pro_8847', 'Med_Famotidine', 'Diag_10_I25', 'Med_Oxycodone', 'Diag_10_Z91', 'Pro_9955', 'Med_Metoclopramide', 'Diag_10_J93', 'Diag_10_E44', 'Med_Tacrolimus', 'Pro_9604', 'Diag_10_E78', 'Diag_10_G20', 'Med_Thiamine', 'Diag_10_D61', 'Med_Levofloxacin', 'Pro_8964', 'Pro_3322', 'Diag_10_I60', 'Pro_9671', 'Diag_10_S01', 'Diag_10_F05', 'Diag_9_9975', 'Pro_3721', 'Diag_10_I31', 'Diag_10_Z23', 'Pro_8938', 'Group_Leiomyoma', 'Diag_9_V462', 'Med_Ranitidine', 'Diag_10_C22', 'Pro_3491', 'Diag_10_R63', 'Pro_3404', 'Diag_10_K70', 'Diag_10_E22', 'Diag_10_J41', 'Med_Prednisone', 'Pro_7359', 'Diag_10_J44', 'Pro_8853', 'Diag_Personal history of antineoplastic chemotherapy', 'Diag_10_G40', 'Diag_10_D66', 'Pro_3613', 'Diag_9_9974', 'Diag_10_K80', 'Diag_10_G89', 'Med_Phenylephrine', 'Med_Ceftazidime', 'Diag_10_S12', 'Pro_387', 'Diag_10_C78', 'Diag_10_G61', 'Med_Furosemide', 'Diag_10_N23', 'Diag_10_E28', 'Diag_10_F30', 'Group_AKI', 'Med_Simvastatin', 'Med_Haloperidol', 'Med_Folic Acid', 'Med_Metronidazole', 'Diag_9_V1251', 'Diag_10_I26', 'Med_Midazolam', 'Pro_0045', 'Med_Magnesium Sulfate', 'Diag_10_N72', 'Med_Atorvastatin', 'Diag_10_C82', 'Med_Lactulose', 'Diag_10_E87', 'Diag_10_I50', 'Diag_10_M81', 'Diag_10_G30', 'Diag_10_R69', 'Diag_10_J43', 'Diag_10_Z87', 'Diag_10_K56', 'Diag_9_V422', 'Diag_10_K92', 'Diag_9_7464', 'Diag_10_F04', 'Diag_10_T78', 'Med_Omeprazole', 'Med_Calcium Gluconate', 'Med_Norepinephrine', 'Med_Ondansetron', 'Diag_10_D50', 'Group_Endometriosis', 'Diag_10_K76', 'Diag_9_V433', 'Pro_3521', 'Diag_10_I61', 'Diag_10_I66', 'Diag_10_C79', 'Pro_4523', 'Med_Dexmedetomidine', 'Diag_10_R10', 'Group_PCOS', 'Diag_10_I20', 'Diag_10_Z95', 'Med_Warfarin', 'Diag_10_I08', 'Diag_9_2113', 'Diag_10_B15', 'Diag_10_N05', 'Diag_10_N13', 'Pro_3611', 'Pro_8744', 'Pro_311', 'Diag_10_R78', 'Pro_9915', 'Diag_10_A40', 'Diag_10_M15', 'Diag_10_G93', 'Diag_10_R58', 'Diag_10_N28', 'Diag_10_K31', 'Diag_10_M06', 'Pro_3950', 'Diag_10_I12', 'Diag_10_F10', 'Med_Sodium Bicarbonate', 'Group_Neoplasm_ovary', 'Med_Nitroglycerin', 'Med_Azithromycin', 'Diag_10_H40', 'Diag_Pneumonia, unspecified organism', 'Diag_10_K66', 'Pro_3522', 'Pro_8162', 'Diag_10_E11', 'Diag_9_E9478', 'Med_Ceftriaxone', 'Diag_10_I62', 'Diag_10_I70', 'Diag_10_G44', 'Diag_10_I33', 'Diag_10_R68', 'Diag_9_7470', 'Diag_10_S06', 'Group_CKD', 'Diag_9_99702', 'Pro_4516', 'Med_Acetaminophen', 'Diag_9_V4365', 'Med_Gabapentin', 'Diag_10_I95', 'Diag_10_H35', 'Diag_10_N39', 'Med_Amiodarone', 'Pro_3324', 'Pro_3612', 'Diag_10_M79', 'Diag_10_F19', 'Pro_3845', 'Diag_10_Z00', 'Diag_9_E9342', 'Pro_4513', 'Pro_5459', 'Diag_10_Z81', 'Pro_9672', 'Diag_10_K85', 'Diag_10_I85', 'Med_Pantoprazole', 'Diag_10_Q20', 'Diag_10_E43', 'Diag_10_I07', 'Pro_3995', 'Diag_10_Z85', 'Pro_3323', 'Diag_9_E8788', 'Diag_9_39891', 'Diag_9_V1259', 'Diag_9_E8786', 'Diag_10_K91', 'Diag_10_Z16', 'Diag_10_A04', 'Diag_10_J84', 'Pro_9390', 'Diag_9_E8780', 'Diag_9_E8497', 'Diag_9_E8781', 'Diag_10_Z51', 'Diag_10_J15', 'Pro_7569', 'Diag_9_E8499', 'Diag_10_I10', 'Diag_10_E89', 'Diag_10_N92', 'Diag_10_F41', 'Pro_9920', 'Diag_10_R19', 'Diag_9_9973', 'Med_Dexamethasone', 'Diag_10_B20', 'Diag_10_K75', 'Med_Insulin', 'Diag_9_64891', 'Med_Lorazepam', 'Pro_4443', 'Pro_3893', 'Pro_9605', 'Diag_10_D70', 'Pro_8051', 'Med_Propofol', 'Diag_10_J45', 'Pro_3723', 'Diag_10_I42', 'Diag_10_O75', 'Pro_8848', 'Diag_9_V440', 'Diag_10_K41', 'Diag_10_F43', 'Diag_10_M60', 'Diag_10_J69', 'Med_Meropenem', 'Diag_9_E9331', 'Diag_10_I24', 'Diag_10_E66', 'Med_Fentanyl Citrate', 'Diag_10_M48', 'Diag_10_T81', 'Diag_10_G43', 'Diag_10_G35', 'Diag_10_G60', 'Pro_8856', 'Med_Aspirin', 'Pro_966', 'Diag_9_V6284', 'Med_Levetiracetam', 'Diag_9_E8490', 'Med_Cefepime', 'Diag_10_T82', 'Pro_3607', 'Diag_10_E83', 'Med_Heparin', 'Med_Linezolid', 'Diag_10_G81', 'Pro_3761', 'Diag_9_V441', 'Pro_3961', 'Diag_9_E8498', 'Diag_9_E8782', 'Diag_10_I69', 'Diag_10_R45', 'Pro_3899', 'Diag_9_5570', 'Pro_9910', 'Pro_3891', 'Pro_9962', 'Diag_Personal history of pulmonary embolism', 'Pro_3897', 'Med_Clindamycin', 'Pro_5491', 'Diag_10_I44', 'Pro_5185', 'Pro_0066', 'Diag_10_Z21', 'Pro_9904', 'Diag_10_K57', 'Pro_4311', 'Diag_10_K59', 'Pro_8622', 'Diag_10_M32', 'Diag_10_F32', 'Pro_0159', 'Med_Ampicillin-Sulbactam', 'Pro_7309', 'Med_Hydromorphone', 'Pro_8855', 'Med_Metoprolol', 'Diag_10_I21', 'Diag_10_M10', 'Diag_10_I71', 'Diag_10_J95', 'Med_Glucagon', 'Diag_Personal history of transient ischemic attack (TIA), and cerebral infarction without residual deficits', 'Diag_9_E8859', 'Pro_8659', 'Diag_9_E8790', 'Pro_8872', 'Diag_10_G47', 'Diag_10_D64', 'Diag_10_I27', 'Diag_10_J98', 'Diag_9_9971', 'Pro_0331', 'Diag_10_K29', 'Pro_0741', 'Diag_10_L03', 'Diag_10_S27']
len(iii_acols)

In [ ]:
print(set(iii_acols)-set(dpm_cols))

In [ ]:
print(set(dpm_cols)-set(iii_acols))

In [ ]:
Final_df = Final_df[~(Final_df[dpm_cols].isna().all(axis=1))]
Final_df.shape

In [ ]:
Final_df[dpm_cols] = Final_df[dpm_cols].fillna(0)

In [ ]:
Final_df.head()

In [ ]:
Final_df.to_csv('/MIMIC4_p_1.csv',index=False)